In [1]:

from arweave.arweave_lib import Wallet, Transaction, API_URL
from arweave.transaction_uploader import get_uploader
import json
import os
from tqdm import tqdm
import cv2

# Load your arweave wallet
your_ar_wallet = Wallet('arweave-key.json')
your_ar_wallet.address, your_ar_wallet.balance

('ApPRG76AL_XAMGkvhuSqIqaSDkeduWBGXA3Cmz4C1Zc', 1.040255040898)

In [2]:
Classes = ["Ruby", "Diamond", "Sapphire", "Emerald", "Serendibite", "Benitoite"]

Report = {}
# for ingl_class in Classes:
#     Report[ingl_class] = []
Report

{}

In [3]:
# for ingl_class in tqdm(Classes):

#     arr = os.listdir(f'./Gems/{ingl_class}')

#     for vid in arr:
#         imgs = cv2.VideoCapture(f'./Gems/{ingl_class}/{vid}')
#         imgs.set(1, 155)
#         ret, frame = imgs.read()
#         cv2.imwrite(f'./Gem_images/{ingl_class}_{vid.split(".")[0]}.jpg', frame)

In [4]:
for ingl_class in tqdm(Classes):

    arr = os.listdir(f'./Gems/{ingl_class}')
    t_dict = {}

    for vid in arr:

        with open(f'./Gems/{ingl_class}/{vid}', 'rb') as file_handler:
            tx = Transaction(your_ar_wallet, file_handler=file_handler, file_path=f'./Gems/{ingl_class}/{vid}')
            tx.add_tag('Content-Type', 'video/mp4')
            tx.sign()

            uploader = get_uploader(tx, file_handler)

            while not uploader.is_complete:
                uploader.upload_chunk()

                # logger.info("{}% complete, {}/{}".format(
                #     uploader.pct_complete, uploader.uploaded_chunks, uploader.total_chunks
                # ))

        vid_url = API_URL+"/"+tx.id

        imgs = cv2.VideoCapture(f'./Gems/{ingl_class}/{vid}')
        imgs.set(1,155)
        ret, frame = imgs.read()
        cv2.imwrite(f'./Gem_images/{ingl_class}_{vid.split(".")[0]}.jpg', frame)

        with open(f'./Gem_images/{ingl_class}_{vid.split(".")[0]}.jpg', 'rb') as f:
            img_in_bytes = f.read()

        transaction = Transaction(your_ar_wallet, data=img_in_bytes)
        transaction.add_tag('Content-Type', 'image/png')
        transaction.sign()
        transaction.send()

        image_url = API_URL+"/"+transaction.id

        metadata = {
        "name": "Ingl-Gem",
        "symbol": "I-Gem",
        "description": "Democratizing The Process of Creating Solana Validators",
        "seller_fee_basis_points": 300,
        "external_url": "https://www.ingl.io",
        "attributes": [
            {
                "trait_type": "Generation",
                "value": "1",
            },
            {
                "trait_type": "Class",
                "value": ingl_class,
            },
            {
                "trait_type": "Rarity",
                "value": vid.split(".")[0],
            },
        ],
        "collection": {
            "name": "Ingl-Gem",
            "family": "Ingl Collections",
        },
        "properties": {
            "files": [
                {
                    "uri": vid_url,
                    "type": "video/mp4",
                },
            ],
            "category": "video",
            "maxSupply": None,
            "creators":[
                {
                "address": "3ksoo5yeKx8GmXA492CL9jkjNFVL8vrAPk818jUftz4J",
                "verified":1,
                "share": 100,
                }
                ],
        },
        "image": image_url,
        }


            # Upload metadata to Arweave
        meta_transaction = Transaction(your_ar_wallet, data=json.dumps(metadata))
        meta_transaction.add_tag('Content-Type', 'text/html')
        meta_transaction.sign()
        meta_transaction.send()

        metadata_url = API_URL+"/"+meta_transaction.id
        t_dict[vid.split(".")[0]] = metadata_url
    Report[ingl_class] = t_dict

100%|██████████| 6/6 [17:32<00:00, 175.49s/it]


In [7]:
with open("uploaded_images.json", "w") as outfile:
    json.dump(Report, outfile)
Report

{'Ruby': {'mythic': 'https://arweave.net/V-GN01-V0OznWUpKEIf0XAMEA_-ndFOfYKNJoPdNpsE',
  'none': 'https://arweave.net/269uSJ8LWFFeWVA44oeJuvAJhD6Otu6_9Ruc323zKi0'},
 'Diamond': {'mythic': 'https://arweave.net/K8A0TAPGY0QZCuyPDXPemGNNtKe97r07JCQ5pS4J6nA',
  'none': 'https://arweave.net/mxqPr11o3xJrV0lSxPpjSjry8YBf2nRGFsPMkiFIL4g'},
 'Sapphire': {'mythic': 'https://arweave.net/00h-GOmxAzRuo3FAyddmd0VVazj6fNId3Xkhg5S3Fww',
  'none': 'https://arweave.net/4ddiMaqN-1LxQuGfcJE3qUbKh-IaULpTK9BYYlbY17s'},
 'Emerald': {'mythic': 'https://arweave.net/RXG9EgRsMVrpGpAd5PgjzCfP0hBdybMepf07wQPoXEU',
  'none': 'https://arweave.net/DdZ9tKy1ZBfTKHFXsPzhkcBMYTSsv5g7bmgZ_d6mG1Y'},
 'Serendibite': {'mythic': 'https://arweave.net/kng_bDJLMewvQ_1_M2YUrRnGJjoH-YWf-eRYeCXj7cE',
  'none': 'https://arweave.net/e3ao7wBo7wSp5EHUNNg79MN8seQTZ5HNCSLQdxTPmlg'},
 'Benitoite': {'mythic': 'https://arweave.net/kvqeN_tHeVprbecix2aO4mBrdHAT8k4szgQxDnu14UQ',
  'none': 'https://arweave.net/Xo-Hk-ZHswayP5kJeiajj1WUCnBnTWH_1_F

In [6]:
# metadata = {
#     "name": "NewB",
#     "symbol": "Ingl-Col",
#     "description": "Democratizing Validator Creation on Solana",
#     "seller_fee_basis_points": 300,
#     "external_url": "https://www.ingl.io",
#     "attributes": [
#         {
#             "trait_type": "Class",
#             "value": "Ruby"
#         }
#     ],
#     "collection": {
#         "name": "ingl-Gem",
#         "family": "Ingl-Collections",
#     },
#     "properties": {
#         "files": [
#             {
#                 "uri": "https://cdn.discordapp.com/attachments/973276746700980224/999358394337087528/0001-0300_2.mp4",
#                 "type": "video/mp4",
#             },
#         ],
#         "category": "video",
#         "maxSupply": 0,
#         "creators":[
#             {
#             "address": "3ksoo5yeKx8GmXA492CL9jkjNFVL8vrAPk818jUftz4J",
#             "verified":1,
#             "share": 100,
#             },
#             ],
#     },
#     "image": "https://cdn.discordapp.com/attachments/948242292794351656/999359930417037372/unknown.png",
# }

# # Upload metadata to Arweave
# meta_transaction = Transaction(your_ar_wallet, data=json.dumps(metadata))
# meta_transaction.add_tag('Content-Type', 'text/html')
# meta_transaction.sign()
# meta_transaction.send()

# metadata_url = API_URL+"/"+meta_transaction.id

# print(metadata_url)